In [22]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

In [27]:
# Load a specific run
run_id = "TEST_llama3_70b_zeroshot-20240528-182337"

path = f"results/{run_id}/df.feather"

import pandas as pd
df = pd.read_feather(path)


In [24]:
# # Load ground truth
# from src.dataset import TDMSDataset, PATH, UNANSWERABLE, LogResult
# valid_dataset = TDMSDataset(PATH.VAL)
# df_ground_truth = pd.DataFrame([{"f":f, "ground_truth":gt} for f, _, gt in [valid_dataset.__getitem__(i) for i in range(len(df))]])
# df = df_ground_truth.merge(df, on="f")

In [34]:
df

,run,f,annontation,ground_truth
0,TEST_llama3_70b_zeroshot-20240528-182337,0806.2287,unanswerable\n,None
1,TEST_llama3_70b_zeroshot-20240528-182337,1412.3670,unanswerable\n,None
2,TEST_llama3_70b_zeroshot-20240528-182337,1001.3451,unanswerable\n,None
3,TEST_llama3_70b_zeroshot-20240528-182337,1907.05888v1,unanswerable\n,None
4,TEST_llama3_70b_zeroshot-20240528-182337,1911.12667v3,unanswerable\n,None
...,...,...,...,...
784,TEST_llama3_70b_zeroshot-20240528-182337,2010.14022v2,unanswerable\n,None
785,TEST_llama3_70b_zeroshot-20240528-182337,1210.0408,unanswerable\n,None
786,TEST_llama3_70b_zeroshot-20240528-182337,1604.03799,unanswerable\n,None
787,TEST_llama3_70b_zeroshot-20240528-182337,0911.1502,unanswerable\n,None


In [26]:
# evaluate
from scoring_program.evaluation import Metrics

results = Metrics.evaluate_property_wise_json_based(
    label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"])
)

results.update(
    Metrics.evaluate_rouge(label_list=list(df["ground_truth"]), prediction_list=list(df["annotation"]))
)
pd.DataFrame([results]).T

KeyError: 'ground_truth'

In [ ]:
# confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(df["ground_truth"], df["pred"])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()